# CNN nhận diện spam email

In [3]:
!pip install tensorflow

   ---------------------------------------- 0.0/331.9 MB ? eta -:--:--
   ---------------------------------------- 2.1/331.9 MB 16.8 MB/s eta 0:00:20
    --------------------------------------- 6.6/331.9 MB 26.9 MB/s eta 0:00:13
   - -------------------------------------- 10.0/331.9 MB 18.3 MB/s eta 0:00:18
   - -------------------------------------- 14.4/331.9 MB 20.6 MB/s eta 0:00:16
   -- ------------------------------------- 24.1/331.9 MB 24.6 MB/s eta 0:00:13
   --- ------------------------------------ 31.5/331.9 MB 26.6 MB/s eta 0:00:12
   ---- ----------------------------------- 39.1/331.9 MB 27.9 MB/s eta 0:00:11
   ----- ---------------------------------- 46.9/331.9 MB 29.0 MB/s eta 0:00:10
   ------ --------------------------------- 54.3/331.9 MB 30.1 MB/s eta 0:00:10
   ------- -------------------------------- 62.4/331.9 MB 30.6 MB/s eta 0:00:09
   -------- ------------------------------- 68.2/331.9 MB 30.4 MB/s eta 0:00:09
   --------- ------------------------------ 76.0/33

In [13]:
!pip install datasets

   ---------------------------------------- 0.0/26.2 MB ? eta -:--:--
   ------ --------------------------------- 4.2/26.2 MB 19.4 MB/s eta 0:00:02
   ------------------ --------------------- 12.1/26.2 MB 28.0 MB/s eta 0:00:01
   ------------------------- -------------- 16.5/26.2 MB 26.6 MB/s eta 0:00:01
   ------------------------------- -------- 20.7/26.2 MB 24.2 MB/s eta 0:00:01
   ---------------------------------------  26.0/26.2 MB 24.9 MB/s eta 0:00:01
   ---------------------------------------  26.0/26.2 MB 24.9 MB/s eta 0:00:01
   ---------------------------------------- 26.2/26.2 MB 20.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 16.1.0
    Uninstalling pyarrow-16.1.0:
      Successfully uninstalled pyarrow-16.1.0


In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# ==============================
# 1. Load dataset (SMS Spam Collection)
# ==============================
url = "https://raw.githubusercontent.com/thehananbhat/spam-vs-ham/main/spam_ham_dataset.csv"
df = pd.read_csv(url)

# Giữ lại 2 cột chính
df = df[['label', 'text']]
df['label'] = df['label'].map({'ham': 0, 'spam': 1})

texts = df["text"].tolist()
labels = df["label"].tolist()

print("Tổng số mẫu:", len(texts))

# ==============================
# 2. Tiền xử lý dữ liệu
# ==============================
max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
X = pad_sequences(sequences, maxlen=max_len, padding="post", truncating="post")

y = np.array(labels)

# Chia train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ==============================
# 3. Xây dựng CNN kiến trúc 
# ==============================
model = Sequential([
    Embedding(input_dim=max_words, output_dim=128, input_length=max_len),
    Conv1D(64, 5, activation="relu"),
    GlobalMaxPooling1D(),
    Dense(2, activation="softmax")  # 2 lớp: spam / ham
])


#=====================================================================
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])


# ==============================
# 4. Train model
# ==============================
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=3,
    batch_size=32,
    verbose=1
)
# ==============================
# 5. Hàm dự đoán email mới
# ==============================
def predict_spam_ham(texts):
    seqs = tokenizer.texts_to_sequences(texts)
    new_X = pad_sequences(seqs, maxlen=max_len, padding="post", truncating="post")
    preds = model.predict(new_X)
    results = []
    for text, pred in zip(texts, preds):
        # pred có 2 giá trị [p_ham, p_spam] vì dùng Dense(2, softmax)
        p_ham, p_spam = pred
        label = 'SPAM 🚨' if p_spam >= 0.5 else 'HAM 👍'
        results.append((text, label, f"{p_spam*100:.2f}% Spam"))
    return results


test_emails = [
    "Win a free iPhone now!!!",
    "Meeting scheduled for tomorrow at 10am",
    "Lowest price discount available, click this link",
    "Can we reschedule our call to next week?"
]

print("\n--- Kết quả Dự đoán ---")
for text, label, prob in predict_spam_ham(test_emails):
    print(f"'{text}'\n -> {label} ({prob})")

Tổng số mẫu: 5171
Epoch 1/3


C:\Users\pokig\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


130/130 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - accuracy: 0.8781 - loss: 0.2668 - val_accuracy: 0.9614 - val_loss: 0.0945
Epoch 2/3
130/130 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.9857 - loss: 0.0533 - val_accuracy: 0.9865 - val_loss: 0.0495
Epoch 3/3
130/130 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.9969 - loss: 0.0158 - val_accuracy: 0.9874 - val_loss: 0.0354

--- Kết quả Dự đoán ---
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
'Win a free iPhone now!!!'
 -> SPAM 🚨 (98.84% Spam)
'Meeting scheduled for tomorrow at 10am'
 -> HAM 👍 (1.04% Spam)
'Lowest price discount available, click this link'
 -> SPAM 🚨 (97.86% Spam)
'Can we reschedule our call to next week?'
 -> HAM 👍 (48.57% Spam)


# CNN nhận diện ảnh

In [1]:
# detect_people_yolo11n.py
# ---------------------------------------------------
# Demo phát hiện người trong ảnh bằng YOLO11n
# Cần cài: pip install ultralytics opencv-python matplotlib
# ---------------------------------------------------

from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import sys

# ================== CONFIG ==================
CONF_THRESH = 0.5   # Ngưỡng confidence (0 → 1). Lớn hơn = ít false positive hơn
MODEL_PATH = "yolo11n.pt"   # model YOLO (n: nano, nhỏ gọn)
# ============================================

def detect_people(image_path):
    # 1. Load model YOLO11n
    model = YOLO(MODEL_PATH)   # sẽ tự tải model lần đầu

    # 2. Chạy dự đoán
    results = model(image_path)

    # 3. Hiển thị ảnh với bounding box
    for r in results:
        im_bgr = r.plot()  # ảnh BGR có vẽ box
        im_rgb = cv2.cvtColor(im_bgr, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10, 10))
        plt.imshow(im_rgb)
        plt.title("Kết quả phát hiện bằng YOLO11n")
        plt.axis("off")
        plt.show()

        # 4. Lọc ra đối tượng class = "person"
        people_boxes = []
        names = r.names
        for box in r.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            if names[cls_id] == "person" and conf >= CONF_THRESH:
                xyxy = box.xyxy[0].tolist()  # [x1, y1, x2, y2]
                people_boxes.append((xyxy, conf))

        print("Số người phát hiện:", len(people_boxes))
        for i, (bbox, conf) in enumerate(people_boxes, 1):
            print(f"Người {i}: BBox={bbox}, Conf={conf:.2f}")

if __name__ == "__main__":
    if len(sys.argv) < 2:
        print("Cách chạy: python detect_people_yolo11n.py <D:/Downloads/vd1.jpg>")
        sys.exit(1)

    image_path = sys.argv[1]
    detect_people(image_path)


ModuleNotFoundError: No module named 'ultralytics'

# CNN kiểu dữ liệu số dự đoán nhiệt độ

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

# Cài đặt hiển thị numpy cho đẹp
np.set_printoptions(precision=2, suppress=True)

temps = 30 + np.random.randn(100)  # trung bình 30°C, thêm nhiễu
print("Nhiệt độ 100 ngày:", temps)

X = temps.reshape(1, 100, 1)  # input shape (batch=1, length=100, feature=1)
y = np.array([30 + np.random.randn()])  # giả lập giá trị ngày 101
print("y:" , y)

model = Sequential([
    Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(100,1)),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')
history = model.fit(X, y, epochs=20, verbose=0)

print("Huấn luyện xong.")
next_day_temp = model.predict(X)
print(f"Dự đoán nhiệt độ ngày kế tiếp: {next_day_temp[0][0]:.2f}°C")
